In [18]:
# !pip install pandas
# !pip install pandasql
# !pip install qiskit-optimization
# !pip install qiskit-algorithms


ERROR: Could not find a version that satisfies the requirement qiskit-primitives (from versions: none)
ERROR: No matching distribution found for qiskit-primitives


In [57]:

import pandas as pd
import numpy as np
import pandasql as ps

aapl_data=pd.read_csv('/content/AAPL.csv',index_col=None)
abb_data=pd.read_csv('/content/ABB.csv',index_col=None)
chl_data=pd.read_csv('/content/CHL.csv',index_col=None)
period=180
# aapl_df=aapl_data[(aapl_data['Date']>'2017-07-31') & (aapl_data['Date']<'2017-08-31')]
# abb_df=abb_data[(abb_data['Date']>'2017-07-31') & (abb_data['Date']<'2017-08-31')]

aapl_query=f'SELECT Date,close,lead(close) over(order by Date) as nxt_close,(lead(close) over(order by Date ASC)-close)/close as aapl_PL from aapl_data limit {period}'
abb_query=f'SELECT Date,close,lead(close) over(order by Date) as nxt_close,(lead(close) over(order by Date ASC)-close)/close as abb_PL from abb_data limit {period}'
chl_query=f'SELECT Date,close,lead(close) over(order by Date) as nxt_close,(lead(close) over(order by Date ASC)-close)/close as chl_PL from chl_data limit {period}'
join_query='SELECT ap.aapl_PL,ab.abb_PL,ch.chl_PL \
          FROM aapl_df ap JOIN abb_df ab on ap.Date=ab.Date \
          JOIN chl_df ch on ap.Date=ch.Date'

abb_df=ps.sqldf(abb_query,locals())
aapl_df=ps.sqldf(aapl_query,locals())
chl_df=ps.sqldf(chl_query,locals())
joined_df=ps.sqldf(join_query,locals())
daily_returns=joined_df.to_numpy()
mu=np.mean(daily_returns,axis=0)
sigma=np.cov(daily_returns.T)
print(sigma)
print(mu)

# kappa={}
# mean_aapl_query='SELECT AVG(aapl_PL) FROM aapl_df'
# mean_abb_query='SELECT AVG(abb_PL) FROM abb_df'
# aapl_kappa=ps.sqldf(mean_aapl_query,locals())
# abb_kappa=ps.sqldf(mean_abb_query,locals())
# kappa['aapl']=abs(aapl_kappa.iloc[0,0])
# kappa['abb']=abs(abb_kappa.iloc[0,0])


[[4.54712234e-04 8.86239722e-05 6.37520560e-05]
 [8.86239722e-05 2.07132817e-04 6.50646729e-05]
 [6.37520560e-05 6.50646729e-05 1.04067696e-04]]
[-0.00207998  0.00152135  0.0001379 ]


In [58]:

def portfolio_cost(x, mu, sigma, x_prev=None, B=None, lambda_=1.0, c=0.01, rho=10.0):
    """
    Computes the total cost of a portfolio configuration.

    Parameters:
    - x: binary vector (numpy array) of current portfolio decisions
    - mu: expected returns vector
    - sigma: covariance matrix
    - x_prev: previous portfolio vector (optional)
    - B: budget constraint (optional)
    - lambda_: risk aversion coefficient
    - c: transaction cost rate
    - rho: budget penalty weight

    Returns:
    - total cost (float)
    """

    # 1. Risk term: λ * xᵀ Σ x
    risk = lambda_ * x.T @ sigma @ x

    # 2. Return term: -μᵀ x
    expected_return = -mu.T @ x

    # 3. Transaction cost: c * sum(|x_i - x_prev_i|)
    if x_prev is not None:
        transaction_cost = c * np.sum(np.abs(x - x_prev))
    else:
        transaction_cost = 0.0

    # 4. Budget penalty: ρ * (sum(x) - B)^2
    if B is not None:
        budget_penalty = rho * (np.sum(x) - B)**2
    else:
        budget_penalty = 0.0

    # Total cost
    total_cost = risk + expected_return + transaction_cost + budget_penalty
    return total_cost

# x = np.array([1, 0])          # current portfolio
# x_prev = np.array([0, 1])     # previous portfolio
# B = 1                            # target: invest in 2 assets

# Compute cost
# cost = portfolio_cost(x, mu, sigma, x_prev, B)
# print("Total Cost:", cost)

def build_qubo(mu, sigma, x_prev=None, B=None, lambda_=1.0, c=0.01, rho=10.0):
    """
    Builds a QUBO matrix from the portfolio cost function.

    Parameters:
    - mu: expected returns vector
    - sigma: covariance matrix
    - x_prev: previous portfolio vector (optional)
    - B: budget constraint (optional)
    - lambda_: risk aversion coefficient
    - c: transaction cost rate
    - rho: budget penalty weight

    Returns:
    - Q: QUBO matrix (numpy array)
    """
    n = len(mu)
    Q = np.zeros((n, n))

    # 1. Risk term: λ * σ
    Q += lambda_ * sigma

    # 2. Return term: -μ → added to diagonal
    for i in range(n):
        Q[i, i] -= mu[i]

    # 3. Transaction cost: c * (1 - 2 * x_prev[i]) → added to diagonal
    if x_prev is not None:
        for i in range(n):
            Q[i, i] += c * (1 - 2 * x_prev[i])

    # 4. Budget penalty: ρ * (sum(x) - B)^2
    # Expand: ρ * (∑x_i^2 + 2∑x_ix_j - 2B∑x_i + B^2)
    for i in range(n):
        Q[i, i] += rho * (1 - 2 * B)
        for j in range(i + 1, n):
            Q[i, j] += 2 * rho
            Q[j, i] += 2 * rho  # ensure symmetry

    return Q

#x_prev = np.array([1, 0])
x_prev=None
B = 3

Q = build_qubo(mu, sigma, x_prev, B)
print("QUBO Matrix:\n", Q)

QUBO Matrix:
 [[-49.9974653   20.00008862  20.00006375]
 [ 20.00008862 -50.00131422  20.00006506]
 [ 20.00006375  20.00006506 -50.00003384]]


In [59]:
import numpy as np
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms import QAOA
# Use V2 Primitives for Qiskit 1.0+
from qiskit.primitives import StatevectorSampler
from qiskit_algorithms.optimizers import COBYLA

n = Q.shape[0] # Number of assets/qubits

# --- 2. Convert the QUBO to a QuadraticProgram object ---
qp = QuadraticProgram()
for i in range(n):
    qp.binary_var(name=f"x{i}")
qp.minimize(quadratic=Q)
print("Quadratic Program created:")
print(qp)

# --- 3. Set up the QAOA algorithm and Simulator ---
# Initialize the V2 StatevectorSampler
sampler = StatevectorSampler()

# Use a classical optimizer
optimizer = COBYLA(maxiter=100)

# Initialize QAOA (Qiskit automatically handles V1/V2 compatibility here)
qaoa_mes = QAOA(sampler=sampler, optimizer=optimizer, reps=3)

# Wrap QAOA in a MinimumEigenOptimizer
qaoa_optimizer = MinimumEigenOptimizer(qaoa_mes)

# --- 4. Run the optimization on the simulator ---
print("\nRunning QAOA on a simulator...")
result = qaoa_optimizer.solve(qp)

# --- 5. Post-process and print results ---
print("\nOptimization completed.")
solution = result.x
cost = result.fval

print(f"Optimal solution (binary vector): {solution}")
print(f"Optimal objective value (minimum cost): {cost}")

selected_assets = [i for i, select in enumerate(solution) if select == 1]
print(f"Selected assets indices: {selected_assets}")


Quadratic Program created:
minimize -49.99746530348416*x0^2 + 40.00017724794442*x0*x1 + 40.00012750411208*x0*x2 - 50.001314219486595*x1^2 + 40.00013012934578*x1*x2 - 50.000033836516586*x2^2 (3 variables, 0 constraints, '')

Running QAOA on a simulator...


/usr/local/lib/python3.12/dist-packages/scipy/sparse/linalg/_dsolve/linsolve.py:606: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
/usr/local/lib/python3.12/dist-packages/scipy/sparse/linalg/_matfuncs.py:707: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  return spsolve(Q, P)
/usr/local/lib/python3.12/dist-packages/scipy/sparse/_index.py:168: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_intXint(row, col, x.flat[0])



Optimization completed.
Optimal solution (binary vector): [0. 1. 1.]
Optimal objective value (minimum cost): -60.001217926657404
Selected assets indices: [1, 2]


In [60]:
import numpy as np

def evaluate_portfolio(x, mu, sigma):
    """
    Computes real-world metrics for a given portfolio.

    Parameters:
    - x: binary vector (numpy array) of selected assets
    - mu: expected returns vector
    - sigma: covariance matrix

    Returns:
    - expected_return: total expected return of the portfolio
    - risk: portfolio variance (risk)
    """
    expected_return = mu.T @ x
    risk = x.T @ sigma @ x
    return expected_return, risk


In [61]:
expected_return, risk = evaluate_portfolio(solution, mu, sigma)
print("Expected Return:", expected_return)
print("Portfolio Risk:", risk)

Expected Return: 0.0016592565155045815
Portfolio Risk: 0.00044132985809732157


In [46]:
print(type(solution))

<class 'numpy.ndarray'>
